In [1]:
// import org.apache.spark.sql.SparkSession
// val spark = SparkSession.builder.master("spark://127.0.0.1:5737").appName("test").getOrCreate()

spark = org.apache.spark.sql.SparkSession@6ad41d04


Spark app-20180927152527-0001: Some(http://192.168.1.102:4042)

In [ ]:
%AddDeps org.vegas-viz vegas_2.11 0.3.11 --transitive --verbose

In [1]:
%AddDeps org.postgresql postgresql 42.2.5

Marking org.postgresql:postgresql:42.2.5 for download
Preparing to fetch from:
-> file:/var/folders/y1/qqvz7wqs5z5_rygrfp97ys5r0000gn/T/toree-tmp-dir6050488973929529089/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/y1/qqvz7wqs5z5_rygrfp97ys5r0000gn/T/toree-tmp-dir6050488973929529089/toree_add_deps/https/repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
-> New file at /var/folders/y1/qqvz7wqs5z5_rygrfp97ys5r0000gn/T/toree-tmp-dir6050488973929529089/toree_add_deps/https/repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.pom


In [371]:
import vegas._
import vegas.data.External._
implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))

render = <function1>


lastException: Throwable = null


<function1>

In [2]:
spark

Spark local-1539348716112: Some(http://192.168.8.102:4040)

In [3]:
spark.conf.getAll

Map(spark.driver.host -> 192.168.8.102, spark.driver.port -> 60660, spark.repl.class.uri -> spark://192.168.8.102:60660/classes, spark.jars -> file:/usr/local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.2.0-incubating.jar, spark.repl.class.outputDir -> /private/var/folders/y1/qqvz7wqs5z5_rygrfp97ys5r0000gn/T/spark-59a329a2-caef-4075-bad2-4283c02bef74/repl-21ece489-8035-4652-a7eb-35a637c4d8ac, spark.app.name -> Apache Toree, spark.executor.id -> driver, spark.submit.deployMode -> client, spark.master -> local[*], spark.app.id -> local-1539348716112)

In [4]:
object spark_read_db {
    val opts = Map("url" -> "jdbc:postgresql://localhost:5733/j3", "driver" -> "org.postgresql.Driver")
    def table(dbtable:String) =
        spark.read.format("jdbc").options(opts + ("dbtable" -> dbtable))
}

defined object spark_read_db


In [5]:
val items = spark_read_db.table("items").load
items.printSchema
items.select("tag").distinct().show()
val matches = spark_read_db.table("match_3c.matches").load
matches.printSchema
val scores = spark_read_db.table("scores").load
scores.printSchema
val roles = spark_read_db.table("roles").load
roles.printSchema
val match_roles = spark_read_db.table("match_3c.match_roles").load
match_roles.printSchema

root
 |-- tag: string (nullable = true)
 |-- id: string (nullable = true)
 |-- content: string (nullable = true)
 |-- inserted_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)

+------------+
|         tag|
+------------+
|       equip|
|metric_names|
|      talent|
|  attr_names|
|     version|
|      kungfu|
+------------+

root
 |-- match_id: long (nullable = true)
 |-- start_time: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- pvp_type: integer (nullable = true)
 |-- map: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- team1_score: integer (nullable = true)
 |-- team2_score: integer (nullable = true)
 |-- team1_kungfu: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- team2_kungfu: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- winner: integer (nullable = true)
 |-- inserted_at: timestamp (nullable = true)
 |-- role_ids: array (nullable = true)
 |    

items = [tag: string, id: string ... 3 more fields]
matches = [match_id: bigint, start_time: int ... 11 more fields]
scores = [role_id: string, match_type: string ... 11 more fields]
roles = [role_id: int, global_id: string ... 10 more fields]
match_roles = [match_id: bigint, role_id: string ... 14 more fields]


[match_id: bigint, role_id: string ... 14 more fields]

In [11]:
import org.apache.spark.sql.functions._

In [7]:
val kungfu_cn_map = Map(
    "xisui" -> ("洗髓", "洗"),
    "yijin" -> ("易经", "秃"),
    "zixia" -> ("紫霞", "气"),
    "taixu" -> ("太虚", "剑"),
    "huajian" -> ("花间", "花"),
    "aoxue" -> ("傲血", "策"),
    "lijing" -> ("离经", "离"),
    "tielao" -> ("铁牢", "铁"),
    "yunshang" -> ("云裳", "秀"),
    "bingxin" -> ("冰心", "冰"),
    "cangjian" -> ("藏剑", "藏"),
    "dujing" -> ("毒经", "毒"),
    "butian" -> ("补天", "补"),
    "jingyu" -> ("惊羽", "鲸"),
    "tianluo" -> ("天罗", "螺"),
    "fenying" -> ("焚影", "明"),
    "mingzun" -> ("明尊", "喵"),
    "xiaochen" -> ("笑尘", "丐"),
    "tiegu" -> ("铁骨", "骨"),
    "fenshan" -> ("分山", "苍"),
    "mowen" -> ("莫问", "莫"),
    "xiangzhi" -> ("相知", "歌"),
    "beiao" -> ("北傲", "霸")
)
val kungfu_items = items.filter("tag == 'kungfu'").
    select($"id".cast("int"), trim($"content", "\"").alias("content")).
    join(kungfu_cn_map.toList.map { case (a, (b, c)) => (a,b,c) }.toDF("content", "kungfu", "short"), Seq("content"), "left_outer")
kungfu_items.show(100)

+--------+-----+------+-----+
| content|   id|kungfu|short|
+--------+-----+------+-----+
|   xisui|10002|    洗髓|    洗|
|   yijin|10003|    易经|    秃|
|   zixia|10014|    紫霞|    气|
|   taixu|10015|    太虚|    剑|
| huajian|10021|    花间|    花|
|   aoxue|10026|    傲血|    策|
|  lijing|10028|    离经|    离|
|  tielao|10062|    铁牢|    铁|
|yunshang|10080|    云裳|    秀|
| bingxin|10081|    冰心|    冰|
|cangjian|10145|    藏剑|    藏|
|  dujing|10175|    毒经|    毒|
|  butian|10176|    补天|    补|
|  jingyu|10224|    惊羽|    鲸|
| tianluo|10225|    天罗|    螺|
| fenying|10242|    焚影|    明|
|xiaochen|10268|    笑尘|    丐|
|   tiegu|10389|    铁骨|    骨|
| fenshan|10390|    分山|    苍|
|   mowen|10447|    莫问|    莫|
|xiangzhi|10448|    相知|    歌|
|   beiao|10464|    北傲|    霸|
+--------+-----+------+-----+



kungfu_cn_map = Map(jingyu -> (惊羽,鲸), yunshang -> (云裳,秀), tiegu -> (铁骨,骨), lijing -> (离经,离), xisui -> (洗髓,洗), fenying -> (焚影,明), xiangzhi -> (相知,歌), taixu -> (太虚,剑), yijin -> (易经,秃), mingzun -> (明尊,喵), cangjian -> (藏剑,藏), tielao -> (铁牢,铁), aoxue -> (傲血,策), fenshan -> (分山,苍), mowen -> (莫问,莫), xiaochen -> (笑尘,丐), butian -> (补天,补), zixia -> (紫霞,气), beiao -> (北傲,霸), bingxin -> (冰心,冰), dujing -> (毒经,毒), tianluo -> (天罗,螺), huajian -> (花间,花))
kungfu_items = [content: string, id: int ... 2 more fields]


error: missing or invalid dependency detected while loading class file 'MetadataBuilder.class'.
Could not access type AnyRef in package scala,
because it (or its dependencies) are missing. Check your build definition for
missing or conflicting dependencies. (Re-run with `-Ylog-classpath` to see the problematic classpath.)
A full rebuild may help if 'MetadataBuilder.class' was compiled against an incompatible version of scala.


[content: string, id: int ... 2 more fields]

In [8]:
matches.groupBy("grade").agg(count("match_id").alias("count"), bround(avg("duration"), 2).alias("duration")).sort("grade").show()

+-----+-----+--------+
|grade|count|duration|
+-----+-----+--------+
| null|  107|  133.26|
|    0|    2|    95.5|
|    1|    7|  148.57|
|    2|    2|    81.0|
|    3|    4|   83.75|
|    4|    2|    25.0|
|    5|    9|   56.33|
|    6|   17|   72.88|
|    7|   32|   92.31|
|    8|   63|   84.08|
|    9|  271|   95.17|
|   10|  954|  102.23|
|   11| 7436|  115.69|
|   12|82572|  130.53|
|   13|37923|   158.1|
+-----+-----+--------+



In [9]:
import org.apache.spark.sql.DataFrame
object Helpers {
implicit class DataFrameHist(df:DataFrame) {
    def mkHistogram(n_bins: Int) = {
        val (startValues, counts) = df.rdd.map(_.getInt(0)).histogram(n_bins)
        (startValues.toList, counts.toList)
    }
}
}
import Helpers._

defined object Helpers


In [10]:
matches.filter("grade==13").select("duration").mkHistogram(50)

(List(2.0, 20.06, 38.12, 56.18, 74.24, 92.3, 110.36, 128.42000000000002, 146.48, 164.54, 182.6, 200.66, 218.72, 236.78, 254.84, 272.9, 290.96, 309.02, 327.08, 345.14, 363.2, 381.26, 399.32, 417.38, 435.44, 453.5, 471.56, 489.62, 507.68, 525.74, 543.8, 561.86, 579.92, 597.98, 616.04, 634.1, 652.16, 670.22, 688.28, 706.34, 724.4, 742.46, 760.52, 778.58, 796.64, 814.7, 832.76, 850.82, 868.88, 886.94, 905.0),List(92, 1011, 2903, 4177, 4572, 4181, 3618, 2959, 2207, 1858, 1620, 1338, 1066, 822, 720, 594, 546, 471, 405, 297, 279, 259, 196, 198, 146, 156, 137, 105, 106, 78, 75, 62, 74, 51, 52, 39, 30, 31, 38, 22, 22, 21, 25, 15, 19, 15, 20, 11, 13, 171))

In [41]:
matches.filter("grade==13").select("duration")

+-------+------------------+
|summary|          duration|
+-------+------------------+
|  count|             37923|
|   mean| 158.0960894444005|
| stddev|126.64352824315579|
|    min|                 2|
|    25%|                80|
|    50%|               120|
|    75%|               191|
|    max|               905|
+-------+------------------+



In [12]:
import org.apache.spark.sql.DataFrame
import scala.collection.JavaConverters._
def get_kungfus(matches:DataFrame) = {
    matches.selectExpr("team1_kungfu as kungfu", "winner=1 as won").
        union(matches.selectExpr("team2_kungfu as kungfu", "winner=2 as won"))
}
object kungfu_sort_fun {
    val kungfu_order = Seq(
        "taixu",
        "zixia", "bingxin", "fenying", "yijin", "huajian", "dujing", "tianluo",  "mowen",
        "aoxue", "fenshan", "cangjian", "jingyu", "xiaochen", "beiao",
        "xisui", "tielao", "tiegu", "mingzun",
        "lijing", "yunshang", "butian", "xiangzhi"
    ).zipWithIndex.toMap
    val kungfu_order_sp = Set(("jingyu", "tianluo"))
    def call(a:String, b:String) : Boolean = {
        if (kungfu_order_sp.contains((a, b))) {
            return true
        }
        if (kungfu_order_sp.contains((b, a))) {
            return false
        }
        return kungfu_order(a) < kungfu_order(b)
    }
}
def show_kungfu(kungfus:DataFrame, n:Int) = {
    val kungfu_map = kungfu_items.select("id", "content").takeAsList(100).asScala.map(i => (i.getAs[Int](0), i.getAs[String](1))).toMap
    val kungfu_short_map = kungfu_items.select("content", "short").takeAsList(100).asScala.map(i => (i.getAs[String](0), i.getAs[String](1))).toMap
    kungfus.takeAsList(n).asScala.map(
        i => (
            i.getAs[Seq[Int]](0).map(kungfu_map).sortWith(kungfu_sort_fun.call).map(kungfu_short_map).mkString,
            i.getAs[Long](1)+1,
            i.getAs[Long](2),
            i.getAs[Long](2).toFloat/i.getAs[Long](1)
        )
    )
}

defined object kungfu_sort_fun


get_kungfus: (matches: org.apache.spark.sql.DataFrame)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
show_kungfu: (kungfus: org.apache.spark.sql.DataFrame, n: Int)scala.collection.mutable.Buffer[(String, Long, Long, Float)]


In [13]:
val kungfu_sort = get_kungfus(matches).groupBy("kungfu")
    .agg(
        count(lit(1)).alias("count"),
        count(when($"won" === true, true)).alias("won")
    ).sort($"won".desc);

kungfu_sort = [kungfu: array<int>, count: bigint ... 1 more field]


[kungfu: array<int>, count: bigint ... 1 more field]

In [14]:
show_kungfu(kungfu_sort, 100).foreach(println)

(鲸丐秀,11070,6192,0.55940014)
(毒毒补,9712,6071,0.6251673)
(剑藏秀,10233,5771,0.56401485)
(策藏秀,11745,5741,0.48884538)
(剑霸秀,7451,4303,0.5775839)
(气花离,6959,3774,0.54239726)
(明莫补,6211,3588,0.5777778)
(秃螺补,4815,2864,0.5949314)
(策鲸秀,4833,2477,0.51262414)
(剑霸补,4635,2372,0.5118688)
(冰毒补,3914,2301,0.5880399)
(冰莫歌,3125,2144,0.6862996)
(毒莫补,3686,2106,0.5715061)
(秃霸补,3540,2013,0.56880474)
(气花补,3611,1903,0.5271468)
(明莫歌,3028,1862,0.6151305)
(剑丐秀,3593,1732,0.48218262)
(毒螺补,2793,1578,0.56518626)
(冰鲸秀,2593,1558,0.60108024)
(苍霸补,2945,1540,0.5230978)
(剑策秀,3070,1538,0.5011404)
(冰鲸离,2489,1401,0.5631029)
(气花秀,2283,1379,0.6042945)
(策藏补,3128,1365,0.43652064)
(苍藏秀,2624,1358,0.5177278)
(剑藏补,2809,1245,0.44337606)
(气鲸离,2159,1223,0.5667285)
(秃霸秀,2060,1190,0.5779505)
(秃秃补,1892,1169,0.6181914)
(剑气离,2086,1116,0.5352518)
(策霸秀,2930,1079,0.3683851)
(明莫离,1930,972,0.503888)
(策霸补,2505,971,0.38777956)
(秃螺秀,1683,952,0.5659929)
(苍霸秀,1921,944,0.49166667)
(冰毒秀,1357,896,0.66076696)
(藏鲸秀,1570,842,0.53664756)
(策鲸补,1718,798,0.46476412)
(

In [15]:
scores.filter($"match_type"==="3c").select("score").mkHistogram(10)

(List(988.0, 1183.8, 1379.6, 1575.4, 1771.2, 1967.0, 2162.8, 2358.6, 2554.4, 2750.2, 2946.0),List(13, 15, 22, 35, 82, 957, 26943, 19377, 796, 37))

In [27]:
val role_kungfu = match_roles.groupBy("role_id", "kungfu").count()
val role_count = role_kungfu.groupBy("role_id").agg(sum("count").alias("total"))
val roles_fileterd = scores.filter($"match_type"==="3c").filter("score>2400")
val kungfu_scores = roles_fileterd.join(role_kungfu, roles_fileterd("role_id") === role_kungfu("role_id"), "left_outer").select("kungfu", "count", "total")

Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`total`' given input columns: [fetch_at, ranking, total_count, updated_at, match_type, kungfu, score, count, inserted_at, score2, mvp_count, ranking2, role_id, win_count, grade, role_id];;
'Project [kungfu#95, count#444L, 'total]
+- AnalysisBarrier
      +- Join LeftOuter, (role_id#43 = role_id#94)
         :- Filter (score#45 > 2400)
         :  +- Filter (match_type#44 = 3c)
         :     +- Relation[role_id#43,match_type#44,score#45,score2#46,grade#47,ranking#48,ranking2#49,total_count#50,win_count#51,mvp_count#52,inserted_at#53,updated_at#54,fetch_at#55] JDBCRelation(scores) [numPartitions=1]
         +- Aggregate [role_id#94, kungfu#95], [role_id#94, kungfu#95, count(1) AS count#444L]
            +- Relation[match_id#93L,role_id#94,kungfu#95,score#96,score2#97,ranking#98,equip_score#99,equip_addition_score#100,max_hp#101,metrics#102,equips#103,talents#104,inserted_at#105,attrs#106,metrics_version#107,attrs_vers

In [18]:
import org.apache.spark.sql.execution.debug._

In [21]:
kungfu_scores.explain

== Physical Plan ==
*(9) Project [kungfu#95, count#279L, total#259L]
+- SortMergeJoin [role_id#43], [role_id#94], LeftOuter
   :- *(2) Sort [role_id#43 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(role_id#43, 200)
   :     +- *(1) Project [role_id#43]
   :        +- *(1) Scan JDBCRelation(scores) [numPartitions=1] [role_id#43] PushedFilters: [*IsNotNull(match_type), *IsNotNull(score), *EqualTo(match_type,3c), *GreaterThan(score,2400)], ReadSchema: struct<role_id:string>
   +- *(8) Project [role_id#94, kungfu#95, count#279L, total#259L]
      +- *(8) SortMergeJoin [role_id#94], [role_id#284], Inner
         :- *(5) Sort [role_id#94 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(role_id#94, 200)
         :     +- *(4) HashAggregate(keys=[role_id#94, kungfu#95], functions=[count(1)])
         :        +- Exchange hashpartitioning(role_id#94, kungfu#95, 200)
         :           +- *(3) HashAggregate(keys=[role_id#94, kungfu#95], functions=[partial_cou

In [19]:
kungfu_scores.debug

Results returned: 11118
== WholeStageCodegen ==
Tuples output: 11118
 kungfu IntegerType: {java.lang.Integer}
 count LongType: {java.lang.Long}
 total LongType: {java.lang.Long}
== Project [kungfu#95, count#279L, total#259L] ==
Tuples output: 0
 kungfu IntegerType: {}
 count LongType: {}
 total LongType: {}
== InputAdapter ==
Tuples output: 0
 role_id StringType: {}
 role_id StringType: {}
 kungfu IntegerType: {}
 count LongType: {}
 total LongType: {}
== SortMergeJoin [role_id#43], [role_id#94], LeftOuter ==
Tuples output: 11118
 role_id StringType: {org.apache.spark.unsafe.types.UTF8String}
 role_id StringType: {org.apache.spark.unsafe.types.UTF8String}
 kungfu IntegerType: {java.lang.Integer}
 count LongType: {java.lang.Long}
 total LongType: {java.lang.Long}
== WholeStageCodegen ==
Tuples output: 10840
 role_id StringType: {org.apache.spark.unsafe.types.UTF8String}
== Sort [role_id#43 ASC NULLS FIRST], false, 0 ==
Tuples output: 0
 role_id StringType: {}
== InputAdapter ==
Tuples o

In [49]:
val tmp = kungfu_scores.select($"kungfu".alias("kungfu_id"), ($"count"/$"total").alias("weight")).groupBy("kungfu_id").agg(sum("weight").alias("weight")).sort($"weight".desc)
tmp.join(kungfu_items, tmp("kungfu_id") === kungfu_items("id")).select("kungfu", "weight").show()

+------+--------------------+
|kungfu|              weight|
+------+--------------------+
|    冰心|   747.8344895091205|
|    太虚|   599.6029102745917|
|    洗髓|0.020833333333333332|
|    分山|               553.0|
|    笑尘|               405.0|
|    紫霞|   597.3970897254083|
|    莫问|   506.9232297938113|
|    傲血|               685.0|
|    天罗|   321.1927706165048|
|    易经|  335.97916666666663|
|    相知|  192.07677020618868|
|    北傲|               686.0|
|    惊羽|   588.8072293834952|
|    离经|   366.5582821330063|
|    补天|   928.2281009343933|
|    云裳|  1053.1655104908796|
|    铁骨|                 1.0|
|    花间|  501.44171786699366|
|    毒经|  414.77189906560704|
|    藏剑|               939.0|
+------+--------------------+
only showing top 20 rows



tmp = [kungfu_id: int, weight: double]


[kungfu_id: int, weight: double]

In [45]:
tmp.explain

== Physical Plan ==
*(11) Sort [weight#1455 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(weight#1455 DESC NULLS LAST, 200)
   +- *(10) HashAggregate(keys=[kungfu_id#1448], functions=[sum(weight#1449)])
      +- Exchange hashpartitioning(kungfu_id#1448, 200)
         +- *(9) HashAggregate(keys=[kungfu_id#1448], functions=[partial_sum(weight#1449)])
            +- *(9) Project [kungfu#349 AS kungfu_id#1448, (cast(count#1257L as double) / cast(total#1237L as double)) AS weight#1449]
               +- SortMergeJoin [role_id#297], [role_id#348], LeftOuter
                  :- *(2) Sort [role_id#297 ASC NULLS FIRST], false, 0
                  :  +- Exchange hashpartitioning(role_id#297, 200)
                  :     +- *(1) Project [role_id#297]
                  :        +- *(1) Scan JDBCRelation(scores) [numPartitions=1] [role_id#297] PushedFilters: [*IsNotNull(match_type), *IsNotNull(score), *EqualTo(match_type,3c), *GreaterThan(score,2400)], ReadSchema: struct<role_id:string>


In [47]:
val roles_fileterd = scores.filter($"match_type"==="3c").filter("score>2400")
val role_matches = roles_fileterd.join(match_roles, Seq("role_id"), "left_outer")
val kungfu_scores =  role_matches.groupBy("role_id", "kungfu").count().join(role_matches.groupBy("role_id").count().withColumnRenamed("count", "total"), "role_id")

roles_fileterd = [role_id: string, match_type: string ... 11 more fields]
role_matches = [role_id: string, match_type: string ... 26 more fields]
kungfu_scores = [role_id: string, kungfu: int ... 2 more fields]


[role_id: string, kungfu: int ... 2 more fields]

In [48]:
kungfu_scores.explain

== Physical Plan ==
*(11) Project [role_id#297, kungfu#349, count#1656L, total#1692L]
+- *(11) SortMergeJoin [role_id#297], [role_id#1695], Inner
   :- *(5) Sort [role_id#297 ASC NULLS FIRST], false, 0
   :  +- *(5) HashAggregate(keys=[role_id#297, kungfu#349], functions=[count(1)])
   :     +- *(5) HashAggregate(keys=[role_id#297, kungfu#349], functions=[partial_count(1)])
   :        +- *(5) Project [role_id#297, kungfu#349]
   :           +- SortMergeJoin [role_id#297], [role_id#348], LeftOuter
   :              :- *(2) Sort [role_id#297 ASC NULLS FIRST], false, 0
   :              :  +- Exchange hashpartitioning(role_id#297, 200)
   :              :     +- *(1) Project [role_id#297]
   :              :        +- *(1) Scan JDBCRelation(scores) [numPartitions=1] [role_id#297] PushedFilters: [*IsNotNull(match_type), *GreaterThan(score,2400), *EqualTo(match_type,3c), *IsNotNull(role_id), ..., ReadSchema: struct<role_id:string>
   :              +- *(4) Sort [role_id#348 ASC NULLS FIRST